# 03 - Benchmark Completo

Este notebook executa o benchmark completo comparando múltiplos modelos LLM.

## Objetivos
1. Testar todos os modelos configurados
2. Comparar todas as estratégias de prompting
3. Avaliar em múltiplos datasets
4. Gerar relatório consolidado

⚠️ **Atenção**: Este notebook pode levar várias horas para executar!

In [1]:
# Configurar path do projeto
import sys
from pathlib import Path
from loguru import logger

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root / "src"))

# Configurar logger
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add(project_root / "reports/logs/03_full_benchmark.log", rotation="10 MB")

logger.info(f"Project root: {project_root}")

2026-01-22 17:26:45.194 | INFO     | __main__:<module>:14 - Project root: /Users/joaoroldi/Projects/tcc2


In [2]:
# Imports
import json
import time
from datetime import datetime
import pandas as pd
import torch
from tqdm.notebook import tqdm

# Módulos do projeto
from config import MODELS, PROMPTING_STRATEGIES, DATASETS, EXPERIMENT_CONFIG, PATHS
from data.data_loader import load_dataset, prepare_test_set
from models.model_handler import ModelHandler
from models.prompts import PromptBuilder
from models.metrics import calculate_metrics, generate_report

# Verificar GPU/MPS
logger.info(f"CUDA disponível: {torch.cuda.is_available()}")
logger.info(f"MPS (Mac) disponível: {torch.backends.mps.is_available()}")

if torch.cuda.is_available():
    logger.info(f"GPU: {torch.cuda.get_device_name(0)}")
    logger.info(f"VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
elif torch.backends.mps.is_available():
    logger.info("Usando Apple Silicon GPU (MPS)")

2026-01-22 17:26:47.052 | INFO     | __main__:<module>:17 - CUDA disponível: False
2026-01-22 17:26:47.061 | INFO     | __main__:<module>:18 - MPS (Mac) disponível: True
2026-01-22 17:26:47.061 | INFO     | __main__:<module>:24 - Usando Apple Silicon GPU (MPS)


## 1. Configuração do Benchmark

In [3]:
# Configurações
SAMPLE_SIZE = 1000  # Número de amostras por dataset
SEED = 42

# Modelos a testar (pode reduzir para teste rápido)
MODELS_TO_TEST = MODELS[:2]  # Primeiros 2 modelos para teste rápido
# MODELS_TO_TEST = MODELS  # Descomentar para testar todos

# Estratégias a testar
STRATEGIES_TO_TEST = PROMPTING_STRATEGIES

# Datasets a testar
DATASETS_TO_TEST = list(DATASETS.keys())

logger.info(f"Modelos: {len(MODELS_TO_TEST)}")
logger.info(f"Estratégias: {len(STRATEGIES_TO_TEST)}")
logger.info(f"Datasets: {len(DATASETS_TO_TEST)}")
logger.info(f"Total de experimentos: {len(MODELS_TO_TEST) * len(STRATEGIES_TO_TEST) * len(DATASETS_TO_TEST)}")

2026-01-22 17:26:47.070 | INFO     | __main__:<module>:15 - Modelos: 2
2026-01-22 17:26:47.070 | INFO     | __main__:<module>:16 - Estratégias: 3
2026-01-22 17:26:47.071 | INFO     | __main__:<module>:17 - Datasets: 2
2026-01-22 17:26:47.071 | INFO     | __main__:<module>:18 - Total de experimentos: 12


## 2. Função de Execução de Experimento

In [4]:
def extract_prediction(response: str) -> int:
    """Extrai predição da resposta do modelo."""
    response_lower = response.lower().strip()
    
    fake_keywords = ["falsa", "fake", "falso", "mentira", "desinformação", "1"]
    true_keywords = ["verdadeira", "verdadeiro", "real", "verídica", "0"]
    
    for keyword in fake_keywords:
        if keyword in response_lower:
            return 1
    
    for keyword in true_keywords:
        if keyword in response_lower:
            return 0
    
    return -1


def run_experiment(model_name, strategy, dataset_name, sample_size, seed):
    """Executa um único experimento."""
    logger.info(f"Experimento: {model_name} | {strategy} | {dataset_name}")
    
    try:
        # 1. Carregar dados
        logger.info("[1/4] Carregando dataset...")
        df = load_dataset(dataset_name)
        test_df = prepare_test_set(df, sample_size=sample_size, seed=seed)
        logger.info(f"      Amostras: {len(test_df)}")
        
        # 2. Carregar modelo
        logger.info("[2/4] Carregando modelo...")
        model_handler = ModelHandler(model_name)
        vram_usage = model_handler.get_vram_usage()
        logger.info(f"      VRAM: {vram_usage:.2f} GB")
        
        # 3. Preparar prompts
        logger.info("[3/4] Preparando prompts...")
        prompt_builder = PromptBuilder(strategy)
        
        if strategy == "few_shot":
            examples_df = df[~df.index.isin(test_df.index)].sample(
                n=EXPERIMENT_CONFIG["few_shot_examples"],
                random_state=seed
            )
            prompt_builder.set_examples(examples_df)
        
        # 4. Executar inferência
        logger.info("[4/4] Executando inferência...")
        predictions = []
        inference_times = []
        
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="      "):
            prompt = prompt_builder.build_prompt(row["text"])
            
            start_time = time.time()
            response = model_handler.generate(prompt)
            end_time = time.time()
            
            prediction = extract_prediction(response)
            predictions.append(prediction)
            inference_times.append(end_time - start_time)
        
        # Calcular métricas
        true_labels = test_df["label"].tolist()
        metrics = calculate_metrics(true_labels, predictions)
        
        metrics["avg_inference_time"] = sum(inference_times) / len(inference_times)
        metrics["total_inference_time"] = sum(inference_times)
        metrics["vram_usage_gb"] = vram_usage
        
        # Resultados
        results = {
            "model": model_name,
            "strategy": strategy,
            "dataset": dataset_name,
            "sample_size": sample_size,
            "timestamp": datetime.now().isoformat(),
            "metrics": metrics,
        }
        
        # Liberar memória
        model_handler.unload()
        torch.cuda.empty_cache()
        
        logger.info(f"✅ Concluído! F1-Score: {metrics['f1_score']:.4f}")
        
        return results
        
    except Exception as e:
        logger.error(f"❌ ERRO: {e}")
        return {
            "model": model_name,
            "strategy": strategy,
            "dataset": dataset_name,
            "error": str(e)
        }

## 3. Executar Benchmark

In [ ]:
# Executar todos os experimentos
all_results = []
results_dir = Path(PATHS['results_dir'])
results_dir.mkdir(parents=True, exist_ok=True)

total_experiments = len(MODELS_TO_TEST) * len(STRATEGIES_TO_TEST) * len(DATASETS_TO_TEST)
experiment_count = 0

logger.info(f"Iniciando benchmark com {total_experiments} experimentos...")

for model_name in MODELS_TO_TEST:
    for strategy in STRATEGIES_TO_TEST:
        for dataset_name in DATASETS_TO_TEST:
            experiment_count += 1
            
            filename = f"{model_name.replace('/', '_')}_{strategy}_{dataset_name}.json"
            filepath = results_dir / filename
            
            # Verificar se já existe resultado de sucesso
            if filepath.exists():
                try:
                    with open(filepath, "r") as f:
                        existing_results = json.load(f)
                        
                    if "error" not in existing_results:
                        logger.info(f"[Experimento {experiment_count}/{total_experiments}] Pular: {model_name} | {strategy} | {dataset_name} (Já concluído)")
                        all_results.append(existing_results)
                        continue
                    else:
                        logger.info(f"[Experimento {experiment_count}/{total_experiments}] Re-executando (erro anterior): {model_name} | {strategy} | {dataset_name}")
                except (json.JSONDecodeError, Exception):
                    logger.info(f"[Experimento {experiment_count}/{total_experiments}] Re-executando (arquivo corrompido): {model_name} | {strategy} | {dataset_name}")

            logger.info(f"[Experimento {experiment_count}/{total_experiments}] Executando: {model_name} | {strategy} | {dataset_name}")
            
            results = run_experiment(
                model_name=model_name,
                strategy=strategy,
                dataset_name=dataset_name,
                sample_size=SAMPLE_SIZE,
                seed=SEED
            )
            
            all_results.append(results)
            
            # Salvar resultados parciais
            with open(filepath, "w") as f:
                json.dump(results, f, indent=2)

logger.info("✅ Benchmark concluído!")

2026-01-22 17:26:47.088 | INFO     | __main__:<module>:9 - Iniciando benchmark com 12 experimentos...
2026-01-22 17:26:47.089 | INFO     | __main__:<module>:26 - [Experimento 1/12] Pular: Qwen/Qwen2-1.5B-Instruct | zero_shot | fakebr (Já concluído)
2026-01-22 17:26:47.089 | INFO     | __main__:<module>:26 - [Experimento 2/12] Pular: Qwen/Qwen2-1.5B-Instruct | zero_shot | fakerecogna (Já concluído)
2026-01-22 17:26:47.090 | INFO     | __main__:<module>:26 - [Experimento 3/12] Pular: Qwen/Qwen2-1.5B-Instruct | few_shot | fakebr (Já concluído)
2026-01-22 17:26:47.090 | INFO     | __main__:<module>:26 - [Experimento 4/12] Pular: Qwen/Qwen2-1.5B-Instruct | few_shot | fakerecogna (Já concluído)
2026-01-22 17:26:47.091 | INFO     | __main__:<module>:26 - [Experimento 5/12] Pular: Qwen/Qwen2-1.5B-Instruct | chain_of_thought | fakebr (Já concluído)
2026-01-22 17:26:47.091 | INFO     | __main__:<module>:26 - [Experimento 6/12] Pular: Qwen/Qwen2-1.5B-Instruct | chain_of_thought | fakerecogna (Já 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps
2026-01-22 17:27:04.356 | INFO     | models.model_handler:_load_model:148 - Modelo carregado com sucesso!
2026-01-22 17:27:04.360 | INFO     | __main__:run_experiment:34 -       VRAM: 0.00 GB
2026-01-22 17:27:04.360 | INFO     | __main__:run_experiment:37 - [3/4] Preparando prompts...
2026-01-22 17:27:04.360 | INFO     | __main__:run_experiment:48 - [4/4] Executando inferência...


      :   0%|          | 0/1000 [00:00<?, ?it/s]

## 4. Gerar Relatório Consolidado

In [ ]:
# Gerar relatório
results_dir = Path(PATHS['results_dir'])
df_report = generate_report(all_results, results_dir)

# Exibir relatório
df_report

## 5. Análise dos Resultados

In [ ]:
# Melhor modelo por F1-Score
best_idx = df_report['f1_score'].idxmax()
best = df_report.loc[best_idx]

logger.info("🏆 MELHOR RESULTADO")
logger.info(f"Modelo: {best['model']}")
logger.info(f"Estratégia: {best['strategy']}")
logger.info(f"Dataset: {best['dataset']}")
logger.info(f"F1-Score: {best['f1_score']:.4f}")
logger.info(f"Acurácia: {best['accuracy']:.4f}")
logger.info(f"Tempo médio: {best['avg_inference_time']:.2f}s")

In [ ]:
# Comparação por estratégia
strategy_comparison = df_report.groupby('strategy')[['accuracy', 'precision', 'recall', 'f1_score']].mean()
strategy_comparison.sort_values('f1_score', ascending=False)

In [ ]:
# Comparação por modelo
model_comparison = df_report.groupby('model')[['accuracy', 'precision', 'recall', 'f1_score']].mean()
model_comparison.sort_values('f1_score', ascending=False)

## 6. Visualizações

Para gerar gráficos detalhados, execute o notebook `04_results_visualization.ipynb`

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráfico de F1-Score por modelo
fig, ax = plt.subplots(figsize=(12, 6))
df_report.groupby('model')['f1_score'].mean().sort_values().plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('F1-Score Médio por Modelo', fontsize=14, fontweight='bold')
ax.set_xlabel('F1-Score')
ax.set_ylabel('Modelo')
plt.tight_layout()
plt.show()

## Conclusões

Documente suas conclusões aqui:
- Qual modelo teve melhor desempenho?
- Qual estratégia de prompting foi mais eficaz?
- Houve diferenças entre os datasets?
- Qual o trade-off entre desempenho e tempo de inferência?